# Plotting Excitation function, raster plot and spike count for a 1D Linear Hawkes process

In [ ]:
# Installation
using Revise
using LinearAlgebra,Statistics,StatsBase,Distributions
using Plots,NamedColors ; theme(:default)
using FFTW
using ProgressMeter
using Random
Random.seed!(0)
using HawkesSimulator; const global H = HawkesSimulator

In [ ]:
function onedmat(x::Real)
  return cat(x;dims=2)
end

In [ ]:
function interaction(t::R,weights_in::AbstractVector{<:R},prepop::H.PopulationState,
    upperbound::Bool) where R
  ret = 0.0
  for (j,w) in enumerate(weights_in)
    train = prepop.trains_history[j]
    ret += H.interaction(t,train,w,prepop.populationtype,upperbound)
  end
  return ret
end

In [ ]:
mytau = 0.5  # kernel time constant
pop = H.PopulationExp(mytau);

In [ ]:
function doplot()
  ts = range(-1.0,5;length=150)
  y = map(t->H.interaction_kernel(t,pop) , ts )
  plot(ts , y,leg=false,xlabel="time (s)", ylabel="interaction kernel", fmt=:png, color="black")
end

doplot()

In [ ]:
# Simulation

function simulate!(networks, num_spikes)
    t_now = 0.0
    H.reset!.(networks) # clear spike trains etc
    @showprogress 1.0 "Running Hawkes process..." for k in 1:num_spikes
        t_now = H.dynamics_step!(t_now, networks)
        if k%1_000 == 0
            H.clear_trains!(networks[1].postpops)
#             H.clear_trains!(networks[2].postpops)
        end # clearing trains after every 1000 spikes
    end
    return t_now
end

In [ ]:
# Initialization
tau_E = 1.0
# tau_I = 10.0

pop_E = H.PopulationExp(tau_E)
# pop_I = H.PopulationExp(tau_I)
w = [0.85]
# w = [1.25 -0.65
#     1.2 -0.5]
# k = 0.03
# w = w .* k

In [ ]:
h = 0.7
baseline_rate = [h]
popstate_E = H.PopulationState(pop_E,baseline_rate)
# popstate_I = H.PopulationState(pop_I, baseline_rate)

# Creating network
# network_E = H.InputNetwork(popstate_E,[popstate_E, popstate_I],[onedmat(w[1,1]),onedmat(w[1,2])])
# network_I = H.InputNetwork(popstate_I,[popstate_E, popstate_I],[onedmat(w[2,1]),onedmat(w[2,2])])
network_E = H.InputNetwork(popstate_E,[popstate_E],[onedmat(w[1,1])])
n_spikes = 90000
# Tmax = simulate!([network_E, network_I],n_spikes)
Tmax = simulate!([network_E],n_spikes)

In [ ]:
numrate_E = H.numerical_rates(popstate_E)
# numrate_I = H.numerical_rates(popstate_I)
# print(numrate_E, numrate_I)
print(numrate_E)

In [ ]:
length(popstate_E.trains_history[1])

In [ ]:
function plot_count(points)
    y = collect(1:length(points))
    plt = plot(xlabel="time (s)", ylabel="count", fmt=:png, legend=:bottomright)
    plot!(plt, points, y, label = "N(t)")
    expected_y = points .* expected_lambda[1]
    plot!(plt, points, expected_y, label = "E[N(t)]")
end

function plot_count(points_E, remean)
    y3 = collect(1:length(points_E))
    plt = plot(xlabel="time (s)", ylabel="Spike Count", legend=:bottomright, fmt=:png)
    plot!(plt, points_E, y3, label = "N_E(t)", color="blue", linetype=:steppost)
end
function plot_count(points_E, points_I, remean, rimean)
    y3 = collect(1:length(points_E))
    plt = plot(xlabel="time (s)", ylabel="Spike Count", legend=:bottomright, fmt=:png)
    plot!(plt, points_E, y3, label = "N_E(t)", color="blue")
    y1 = zeros(length(points_E))
    y2 = zeros(length(points_I))
    for i in 1:length(points_E)
        y1[i] = remean*(points_E[i]-points_E[1])
    end
    for i in 1:length(points_I)
        y2[i] = rimean*(points_I[i]-points_I[1])
    end
    y4 = collect(1:length(points_I))
    plot!(plt, points_I, y4, label = "N_I(t)", color="red")
    plot!(plt, points_E,y1, label="E[N_E(t)]", color="dark blue")
    plot!(plt, points_I,y2, label="E[N_I(t)]", color="dark red")
end

In [ ]:
function rasterplot(tlims = (20.,50) )
  _trainE = popstate_E.trains_history[1]
  plt=plot()
  trainE = filter(t-> tlims[1]< t < tlims[2],_trainE)
  nspk = length(trainE)
  scatter!(plt,trainE,fill(0.5,nspk),markersize=50, markercolor=:blue,markershape=:vline,legend=:topright, label="Excitatory")
#   _trainI = popstate_I.trains_history[1]
#     trainI = filter(t-> tlims[1]< t < tlims[2],_trainI)
#   nspk = length(trainI)
#   scatter!(plt,trainI,fill(1,nspk),markersize=35, markercolor=:red,markershape=:vline,legend=:topright, label="Inhibitory")
  plot!(plt,ylims=(0,1),xlabel="time (s)",fmt=:png)
end

rasterplot()

In [ ]:
plot_count(popstate_E.trains_history[1][82:170], numrate_E[1])

In [ ]:
function findrate(t_now::Real,inp::H.InputNetwork,ineu::Integer;upperbound::Bool=false)
  ret = 0.0
  for (w,prepop) in zip(inp.weights,inp.prepops)
    w_in = view(w,ineu,:)
    ret += interaction(t_now,w_in,prepop,upperbound)
  end
  return ret
end

In [ ]:
# Excitation Function vs Time
function  plot_excitation(networks, tlims=(0,1500))
    num_neurons = size(networks,1)
#     trainsh = [networks[1].postpops.trains_history[1], networks[2].postpops.trains_history[1]]
    trainsh = [networks[1].postpops.trains_history[1]]
    global expected_lambda = H.numerical_rate.(trainsh)
    times = range(tlims...;length=100)
    for i in 1:num_neurons
        trains = filter(t-> tlims[1]<=t<=tlims[2],trainsh[i])
        times = sort(vcat(times, trains))
    end
    plt = plot(xlabel="time (s)", ylabel="Rate (Hz)", fmt=:png, legend=:bottomright)
    y_vector = Array{Float64}(undef, length(times), num_neurons)
    expected_lambda_vector = zeros(Float64, length(times), num_neurons)
    for i in 1:num_neurons
        y_vector[:,i] = map(t->findrate(t,networks[i],1;upperbound=false), times)
#         println(y_vector[:,1])
        y_vector[:,i] = y_vector[:,i] .+ baseline_rate[1]
        expected_lambda_vector[:,i] = expected_lambda_vector[:,i] .+ expected_lambda[i]
    end
    plot!(plt, times , y_vector[:,1], label = "rate_E(t)", color="blue")
#     plot!(plt, times , y_vector[:,2], label = "rate_I(t)", color="red")
    plot!(plt, times , expected_lambda_vector[:,1], label = "E[rate_E(t)]", color="dark blue", linestyle=:dash)
#     plot!(plt, times , expected_lambda_vector[:,2], label = "E[rate_I(t)]", color="dark red")
end

In [ ]:
plot_excitation([network_E], (20, 50))